## Assignment 08 : Colin Kirby

In [1]:
# Imports.
import pygame
import moderngl
import glm
import numpy as np
from LoadObject import getObjectData

pygame 2.6.1 (SDL 2.28.4, Python 3.13.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Initialize Pygame & OpenGL
def init_window(width=800, height=600):
    pygame.init()
    pygame.display.set_mode((width, height), pygame.OPENGL | pygame.DOUBLEBUF)
    pygame.display.set_caption("Assignment 08 : Colin Kirby")

    # Create and Set up OpenGL Context.
    ctx = moderngl.create_context()
    ctx.enable(moderngl.CULL_FACE)
    ctx.enable(moderngl.DEPTH_TEST)

    return ctx

In [3]:
# Initialize Window.
ctx = init_window()

In [4]:
# Load the Vertex Data for Platform, Teapot, and Light Source.
vertex_data_platform, _ = getObjectData("cube.obj", normal=True, texture=True)
vertex_data_teapot, _ = getObjectData("teapot_with_texCoords.obj", normal=True, texture=True)
vertex_data_light, _ = getObjectData("20_icosahedron.obj", normal=True)

Normal exists
texture exists
Normal exists
texture exists
Normal computed.


In [5]:
# Global VAO and Program references.
vao_platform = None
vao_teapot = None
vao_light = None
program_platform = None
program_teapot = None
program_light = None

In [6]:
# Texture loading function.
def load_texture(filename):
    texture_img = pygame.image.load(filename)
    texture_data = pygame.image.tobytes(texture_img, "RGB", True)
    texture = ctx.texture(texture_img.get_size(), 3, texture_data)
    texture.build_mipmaps()  # Enable mipmaps
    return texture

In [7]:
# Load textures for Grass (Floor) and Gold (Teapot).
texture_floor = load_texture("grass.jpg")
texture_teapot = load_texture("gold.jpg")

In [8]:
# Upload Vertex Data for Platform.
def set_vert_data_platform():
    global vao_platform, program_platform

    # Convert Vertex Data to NP Array.
    vertex_data_platform_np = np.array(vertex_data_platform, dtype='f4')
    vbo_platform = ctx.buffer(vertex_data_platform_np.tobytes())

    # Compile Shader Program.
    program_platform = ctx.program(
        # Vertex Shader
        vertex_shader=""" 
        #version 330
        in vec3 in_position;
        in vec3 in_normal;
        in vec2 in_uv;

        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 proj;
        uniform mat3 normalMatrix;

        out vec3 frag_pos;
        out vec3 frag_normal;
        out vec2 frag_uv;

        void main() {
            frag_pos = vec3(model * vec4(in_position, 1.0));
            frag_normal = normalize(normalMatrix * in_normal);
            frag_uv = in_uv;
            gl_Position = proj * view * vec4(frag_pos, 1.0);
        }
        """,
        # Fragment Shader for Platform.
        fragment_shader="""
        #version 330
        in vec3 frag_pos;
        in vec3 frag_normal;
        in vec2 frag_uv;

        uniform vec3 light_pos;
        uniform vec3 light_direction;
        uniform bool is_point_light;
        uniform bool metal; // Include metal uniform
        uniform sampler2D texture_diffuse;

        out vec4 frag_color;

        void main() {
            vec3 tex_color = texture(texture_diffuse, frag_uv).rgb;
            vec3 norm = normalize(frag_normal);
            vec3 light_dir = is_point_light ? normalize(light_pos - frag_pos) : normalize(-light_direction);

            if (metal) {
                // Specular computation (not used for floor)
                frag_color = vec4(0.0); // Could set to black or handle appropriately
            } else {
                float diff = max(dot(norm, light_dir), 0.0);
                vec3 diffuse = diff * tex_color;
                frag_color = vec4(diffuse, 1.0);
            }
        }
        """
    )

    # Create the VAO for the Platform.
    vao_platform = ctx.vertex_array(
        program_platform,
        [
            (vbo_platform, '3f 3f 2f', 'in_position', 'in_normal', 'in_uv')
        ]
    )

In [9]:
# Upload Vertex Data for Teapot.
def set_vert_data_teapot():
    global vao_teapot, program_teapot

    # Convert Teapot Vertex Data to NP Array.
    vertex_data_teapot_np = np.array(vertex_data_teapot, dtype='f4')
    vbo_teapot = ctx.buffer(vertex_data_teapot_np.tobytes())

    # Compile Shader Program.
    program_teapot = ctx.program(
        # Vertex Shader
        vertex_shader="""
        #version 330
        in vec3 in_position;
        in vec3 in_normal;
        in vec2 in_uv;

        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 proj;
        uniform mat3 normalMatrix;

        out vec3 frag_pos;
        out vec3 frag_normal;
        out vec2 frag_uv;

        void main() {
            frag_pos = vec3(model * vec4(in_position, 1.0));
            frag_normal = normalize(normalMatrix * in_normal);
            frag_uv = in_uv;
            gl_Position = proj * view * vec4(frag_pos, 1.0);
        }
        """,
        # Fragment Shader for Teapot.
        fragment_shader="""
        #version 330
        in vec3 frag_pos;
        in vec3 frag_normal;
        in vec2 frag_uv;

        uniform vec3 light_pos;
        uniform vec3 light_direction;
        uniform vec3 eye_pos;
        uniform bool is_point_light;
        uniform bool metal;
        uniform sampler2D texture_diffuse;
        uniform float shininess;

        out vec4 frag_color;

        void main() {
            vec3 tex_color = texture(texture_diffuse, frag_uv).rgb;
            vec3 norm = normalize(frag_normal);
            vec3 light_dir = is_point_light ? normalize(light_pos - frag_pos) : normalize(-light_direction);

            if (metal) {
                // Specular computation using Blinn-Phong model
                vec3 view_dir = normalize(eye_pos - frag_pos);
                vec3 halfway_dir = normalize(light_dir + view_dir);
                float spec = pow(max(dot(norm, halfway_dir), 0.0), shininess);

                vec3 specular = spec * tex_color;
                frag_color = vec4(specular, 1.0);
            } else {
                // Diffuse computation (not used for teapot in this assignment)
                float diff = max(dot(norm, light_dir), 0.0);
                vec3 diffuse = diff * tex_color;
                frag_color = vec4(diffuse, 1.0);
            }
        }
        """
    )

    # Create VAO for Teapot.
    vao_teapot = ctx.vertex_array(
        program_teapot,
        [
            (vbo_teapot, '3f 3f 2f', 'in_position', 'in_normal', 'in_uv')
        ]
    )

In [10]:
# Upload Vertex Data for Light Source.
def set_vert_data_light():
    global vao_light, program_light

    # Convert Light Vertex Data to NP Array.
    vertex_data_light_np = np.array(vertex_data_light, dtype='f4')
    vbo_light = ctx.buffer(vertex_data_light_np.tobytes())

    # Compile Light Shader Program.
    program_light = ctx.program(
        vertex_shader="""
        #version 330
        in vec3 in_position;

        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 proj;

        void main() {
            gl_Position = proj * view * model * vec4(in_position, 1.0);
        }
        """,
        fragment_shader="""
        #version 330
        out vec4 frag_color;

        void main() {
            frag_color = vec4(1.0); // White light source
        }
        """
    )

    # Create VAO for Light Source.
    vao_light = ctx.vertex_array(
        program_light,
        [
            (vbo_light, '3f', 'in_position')
        ]
    )

In [11]:
# Initialize Vertex Data.
set_vert_data_platform()
set_vert_data_teapot()
set_vert_data_light()

In [12]:
# Projection Matrix.
fov = 60.0  
width, height = 800, 600
aspect_ratio = width / height  
near_plane = 10.0  
far_plane = 45.0 
proj_matrix = glm.perspective(glm.radians(fov), aspect_ratio, near_plane, far_plane)


In [13]:
# Static Camera Position.
eye_position = glm.vec3(20.0, 10.0, 0.0)
view_matrix = glm.lookAt(
    eye_position,            # Camera position
    glm.vec3(0.0, 0.0, 0.0), # Look at origin
    glm.vec3(0.0, 1.0, 0.0)  # Up Vector
)

In [ ]:
# Rendering Loop.
running = True
is_point_light = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN and event.key == pygame.K_l:
            is_point_light = not is_point_light

    # Clear the screen at the beginning of each frame.
    ctx.clear(0.1, 0.1, 0.1, depth=1.0)

    # Time & Light Position for Orbiting Light.
    time_in_seconds = pygame.time.get_ticks() / 1000.0
    if is_point_light:
        light_pos = glm.vec3(15.0 * glm.cos(time_in_seconds), 15.0, 15.0 * glm.sin(time_in_seconds))
        light_direction = glm.vec3(0.0, -1.0, 0.0)
    else:
        angle = time_in_seconds
        light_direction = glm.normalize(glm.vec3(glm.cos(angle), -1.0, glm.sin(angle)))
        light_pos = glm.vec3(0.0, 0.0, 0.0)

    # Render Platform w/ Grass.
    if vao_platform is not None and program_platform is not None:
        texture_floor.use(location=0)
        program_platform['texture_diffuse'].value = 0
        program_platform['metal'].value = False

        # Set matrices & uniforms.
        model_matrix_platform = glm.scale(glm.mat4(1.0), glm.vec3(10.0, 0.05, 10.0))
        normal_matrix_platform = glm.transpose(glm.inverse(glm.mat3(model_matrix_platform)))

        program_platform['model'].write(np.array(model_matrix_platform, dtype='f4').T.flatten().tobytes())
        program_platform['normalMatrix'].write(np.array(normal_matrix_platform, dtype='f4').T.flatten().tobytes())
        program_platform['view'].write(np.array(view_matrix, dtype='f4').T.flatten().tobytes())
        program_platform['proj'].write(np.array(proj_matrix, dtype='f4').T.flatten().tobytes())
        program_platform['light_pos'].write(np.array(light_pos, dtype='f4').tobytes())
        program_platform['light_direction'].write(np.array(light_direction, dtype='f4').tobytes())
        program_platform['is_point_light'].value = is_point_light

        vao_platform.render(moderngl.TRIANGLES)

    # Render Teapot w/ Gold.
    if vao_teapot is not None and program_teapot is not None:
        texture_teapot.use(location=0)
        program_teapot['texture_diffuse'].value = 0
        program_teapot['metal'].value = True
        program_teapot['eye_pos'].write(np.array(eye_position, dtype='f4').tobytes())
        program_teapot['shininess'].value = 25.0

        # Apply rotation, scaling, and translation to the teapot.
        rotation_matrix_teapot = glm.rotate(glm.mat4(1.0), glm.radians(90.0), glm.vec3(0.0, 1.0, 0.0))
        model_matrix_teapot = glm.scale(rotation_matrix_teapot, glm.vec3(0.3))
        model_matrix_teapot = glm.translate(model_matrix_teapot, glm.vec3(0.0, 7.875, 0.0))
        normal_matrix_teapot = glm.transpose(glm.inverse(glm.mat3(model_matrix_teapot)))

        program_teapot['model'].write(np.array(model_matrix_teapot, dtype='f4').T.flatten().tobytes())
        program_teapot['normalMatrix'].write(np.array(normal_matrix_teapot, dtype='f4').T.flatten().tobytes())
        program_teapot['view'].write(np.array(view_matrix, dtype='f4').T.flatten().tobytes())
        program_teapot['proj'].write(np.array(proj_matrix, dtype='f4').T.flatten().tobytes())
        program_teapot['light_pos'].write(np.array(light_pos, dtype='f4').tobytes())
        program_teapot['light_direction'].write(np.array(light_direction, dtype='f4').tobytes())
        program_teapot['is_point_light'].value = is_point_light

        vao_teapot.render(moderngl.TRIANGLES)

    # Render Light Source.
    if is_point_light and vao_light is not None and program_light is not None:
        # Scale & Translate Light Source Model.
        model_matrix_light = glm.translate(glm.mat4(1.0), light_pos) * glm.scale(glm.mat4(1.0), glm.vec3(0.1))

        program_light['model'].write(np.array(model_matrix_light, dtype='f4').T.flatten().tobytes())
        program_light['view'].write(np.array(view_matrix, dtype='f4').T.flatten().tobytes())
        program_light['proj'].write(np.array(proj_matrix, dtype='f4').T.flatten().tobytes())

        vao_light.render(moderngl.TRIANGLES)

    # Swap Display Buffers.
    pygame.display.flip()

# Clean Up.
pygame.quit()
